In [1]:
import os
import math
import copy
import time
from os.path import exists
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


In [2]:
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [5]:
#Some convenient helper functions

def is_interactive_notebook():
    return __name__ == "__main__"

def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr" : 0}]
        None
    
    def step(self):
        None
    
    def zero_grad(self, set_to_none=False):
        None
        
class DummyScheduler:
    def step(self):
        None

The goal of reducing sequential computation also forms the foundation of the extended Neural GPU, ByteNet, and ConvS2S, all of which use convolutional neural networks as a basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input and output positions grows in the distance between positions. This makes it more difficult to learn dependencies between distant positions. In the Transformer, this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention.

Self-attention, sometimes called intra-attention, is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment, and learning task-independent sentence representations. End-to-end memory networks are based on a recurrent attention mechanism instead of a sequence-aligned recurrence and have been shown to perform well on simple-language question answering and language modelling tasks.

To the best of our knowledge however, the Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence aligned RNNs or convolutions.

## **Part 1. Model Architecture**


Most competitive neural sequence transduction models have an encoder-decoder structure. Here, the encoder maps an input sequence of symbol representations $(x_1,\ldots, x_n)$ to a sequence of continuous representations $z = (z_1,\ldots, z_n)$. Given $z$, the decoder then generates an output sequence $(y_1,\ldots, y_m)$ of symbols one element at a time. At each step, the model is auto-regressive, consuming the previously generated symbols as additional input when generating the text.